# Project 1: Navigation
Several architectures and hyper-parameter setups were considered for solving the problem. All of them reached the
average score of 13+ on 100 consecutive runs, so we will focus here on comparing the most prominent setups based on
their top scores and how many episodes they needed to solve the navigation problem.

## Q-Network architectures
### Comparing the performance of some architectures
In the core of each agent was a network which estimated the Q-function. All networks were fully connected networks
consisting solely of linear layers and ReLU activations. Batch normalization was briefly tested, but discarded because
it didn't offer any improvement. A couple of different setups were considered and tested on 2000 episodes. Their
performances were very comparable as one can see in the following figure:
![Comparison of network architectures](artifacts/architecture_comparison.png)
- The numbers separated by `_` designate the
width of each dense layer of the network
- The vertical dashed lines mark the first episode where the average score surpassed 13
- The reward values are smoothened with a window of legth 100 episodes.

Based on the similarity of the results, the network with three hidden layers of widths (64, 64, 64) was picked in order
to make sure the performance of setups to be tested is not hindered by a lack of network capacity. This architecture
was fix throughout all of the experiments described below.

### Dueling DQN
A second network is defined which implements the dueling DQN architecture. The output of the last 64 width layer of the
(64, 64, 64) network mentioned above is used as input to 2 dense networks which compute respectively the value of the
input state and the advantage of the state-action combination. The Q function is then computed as follows:

$Q(s, a) = V(s) + (A(s, a) - E_{a'\in\mathcal{A}}[A(s, a')])$

Both the A and V networks have two layer with sizes (64, 32) and they output respectively $|\mathcal{A}|$ and 1
values.

The rest of the training process of the network remains the same and the premise is that the agent will learn both
long-term strategies leading to better values (e.g. moving to areas with higher yellow to blue banana ration) and
short-term strategies (e.g. collecting yellow bananas and avoiding blue ones), leading to a better performance.

For more on dueling DQN, look at the [paper which introduced the method](https://arxiv.org/abs/1511.06581).

## Network Training setup
### Hyper parameters
The hyper parameters directly related to the network training were the following:
- learning rate: Check the section below
- batch size: This was fixed to 64
- hidden layers: This set the number of hidden layers of the backbone fully connected network and their with. After a
    short experimentation was fixed to (64, 64, 64)
- dueling_dqn: Flag to toggle use of the dueling DQN architecture
    
### Learning rate
It was initially set to 1e-5 but after a couple of experiments, it was evident that the network would benefit from a
higher value, at least on the first episodes. This let to using the relatively high rate of 5e-4 in the beginning of
training and later on decreasing it on plateaus with a scheduler provided by PyTorch. This step help stabilize a bit
the progress of learning on the last steps of training.

## Q-Learning setup
### Hyper parameters
The following hyper parameters were used for the Q-learning parts not directly related to the network:
- number_of_episodes: This was fixed to 2000 for all experiments and set to 4000 for a final comparion of DQN and its extensions
- maximum_timestaps: 1000
- initial_epsilon: The exploration rate started from 1.0
- final_epsilon: Then decayed to 0.01
- epsilon_decay: With a decay rate of 0.995
- double_dqn: Flag to toggle use of double DQN
- prioritize_replay: Flag to toggle use of priority experience replay
- per_alpha: In case prioritized replay is on, set its constant alpha
- per_beta_0: In case prioritized replay is on, set the initial value of $\beta$. $\beta$ will decay linearly
    starting from $\beta_0$ on the first episode and becoming 1.0 on the last episode.
    
### Double DQN
Double DQN applies a small change to the equation used to estimate the total reward using the Q function. The change
is supposed to compensate the overestimation of values given that the max value action is selected for the next step.
To compensate for that the index of the max is computed by the local network, while the value by the target network.
The equation computing the expected value is:

$Y_t = R_t + \gamma Q(s_{t + 1}, argmax_{a \in \mathcal{A}}(Q(s_{t + 1}, a; \theta)); \theta^-)$

where $\theta$ are the parameters of the local network, while $\theta^-$ the parameters of the target network.

For more on double DQN, look at the [paper which introduced the method](https://arxiv.org/abs/1509.06461).

### Prioritized experience replay
Prioritized replay controls how the examples used to train the model are picked. Namely, a probability function defined
over all examples gathered in the replay buffer is used and assigns higher probabilities to examples whose Q values
estimated using actual rewards deviate more from the Q values computed directly from the Q-network. The implementation
involved the introduction of a new buffer which stores priorities along with updates based on the aforementioned errors
and also samples based on the distribution defined by them.

In order to make sampling efficient, one has to introduce an efficient structure such as a sum tree. This was avoided
here in order to save time and was not a real problem given that the number of episodes was fairly low. A couple of
cheap caching tricks (mainly on the calculation of the maximum priority) were enough to make the implementation 
fast enough.

For more on prioritized replay, look at the [paper which introduced the method](https://arxiv.org/abs/1511.05952).

### CPU vs GPU
Given the small size of the networks used and the fact there is a constant conversion from tensors to numpy arrays,
it is faster training on cpu, probably because of avoiding intermediate translation steps. If a version of Unity
native to PyTorch were available, then probably one could get significant gains using gpus. Of course, in the case
where the neural network has a non-trivial architecture e.g. a modern CNN used to learn from pixels, then this
overhead would be small compared to the network training time.

## Comparison of DQN with extensions
Once the backbone network architecture and learning rate were fixed a series of experiments was run on 4000 episode each to evaluate the effect of adding common improvements to the DQN network.
The experiment can be thought as a baby version of rainbow, which attempts to analyze the added value when combining several extensions of DQN.

### Setups
The following experiments were run:
- DQN: The initial DQN architecture
- Dueling DQN: The dueling DQN architecture
- Double, dueling DQN: The dueling DQN with a double expected value function definition
- Double, dueling per DQN: All the above using additionally prioritized experience replay

### Reward comparison
Those are the training reward curves of the above experiments:

![Comparison of DQN extensions](artifacts/dqn_improvements_comparison.png)

### Conclusions
All cominations of improvements exceed the score of 13, but none of them seems to provide a significant advantage in the speed of learning or in the long term performance which is around 17.5. Combined with the fact that the agents behave in a near optimal way when tested live on the environment, this indicates that the problem is too simple for the improvements of DQN to demonstrate an advantage.

## Example test run of a trained DQN agent
Below we can see a giff with the behavior of a train agent using a dqn network. This is a randomly selected environment which is not cherry-picked. To reproduce, one can run the following script:

`python scripts/test_agent_in_environment.py --agent-parameters-path experiments/final_comparison_dqn/checkpoint.pth --no-dueling-dqn`

![Agent test run](artifacts/screencast_unity_edited.gif)

## Future ideas to improve performance
Here are some ideas on how to improve performance.

### Error analysis
This would be the first step in order to figure out what is wrong in the agent's behaviour. One could e.g. test the agent on several episodes and watch the
worst ones to figure out what went wrong. If it is a lack of planning, one could look of improvements which help the agent plan better or trying out a deeper
network which has more capacity or even adding some memory to the agent (e.g. use an RNN). If on the other hand the agent has problems on short-term actions,
one could look into methods similar to dueling DQN which add extra focus on that. Finally, it could be the case (which actuall seems like that) that the agent
has a near optimal behavior and it is not worth it trying to further optimize its training.

In any case, the error analysis would give a direction to the next investigations and also a feeling of how hard/easy each direction could be.

### Use a higher level framework
It is a nice exercise writing all those algorithms from scratch, but if one where to focus on optimizing such systems, one should turn to some popular framework
in the field of reinforcement learning which has most SOTA algorithms implemented. 

Some popular frameworks seem to be:
- [ReAgent](https://reagent.ai/)
- [Dopamine](https://github.com/google/dopamine/tree/master/docs)

### Rainbow
Rainbow combines most of the recent proposed reinforcement learning methods in one algorithm and seems to be a quite sucessful and established architecture.
It could be a good candidate to apply to this problem.

### Learn from pixels
If the error analysis shows that there are errors due to lack of information in the ray based perception, one could try getting the image input and train a model
from pixels.